[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/v-artur/Golden_Oreos/blob/main/Modeling.ipynb)

Import files from drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Spectrogram reconstruction 

# 1.) One person model

Functions for modeling and predicting

In [2]:
import tensorflow as tf
from tensorflow.keras import regularizers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, TimeDistributed
import numpy as np

# Bottleneck FC-DNN

def create_ae_model(inputsize, outputsize):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(inputsize)))
    model.add(tf.keras.layers.Dense(128, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(64, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(16, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(4, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(16, activation="relu", kernel_initializer='HeNormal'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(outputsize))
    return model


# LSTM/GRU Network

def create_LSTM_model(inputsize, outputsize):
  model = tf.keras.Sequential()
  model.add(GRU(units=300, dropout=0.2, return_sequences=True, input_shape=(inputsize,1)))
  model.add(GRU(units=300, dropout=0.2, return_sequences=True, input_shape=(inputsize,1)))
  model.add(Flatten())
  model.add(Dense(outputsize))
  return(model)


# CNN network
def create_cnn_model(rows_per_feature, cols_per_feature, outputsize):
  model = tf.keras.Sequential()
  model.add(Conv1D(10, 8, activation="relu", input_shape=(rows_per_feature, cols_per_feature,)))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(10, 20, activation="relu"))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(units=outputsize, activation='linear'))
  return(model)

# to make a 2D object out of the EEG featurevector with sliding window
# windowsize is minimum 10 and multiple of 2.
def cnnize(data, window):
  data = np.asarray(data)
  #the new array, the first coordinate is the number of samples,
  #the second and the third are the new features
  X = np.zeros((data.shape[0], int((data.shape[1]-window)/2)+1 , window))
  for index, elem in enumerate(data):
    # creating the feature-parts
    new_elems = np.array([elem[i:i+window] for i in range(0, data.shape[1] - window + 1, 2)])
    X[index] = new_elems
  return X


In [3]:
# setting seed
tf.keras.utils.set_random_seed(1234)

### Optimizing the FC-DNN structure for one person



In [4]:
from scipy.stats import pearsonr
from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



data = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_feat.npy')
spectrogram = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_spec.npy')

#Inital parameters
nfolds = 10
kf = KFold(nfolds,shuffle=False)
pca = PCA()
numComps = 200
explainedVariance = np.zeros(nfolds)
val_split = 0.2


#Initialize an empty spectrogram to save the reconstruction to
rec_spec = np.zeros(spectrogram.shape)
#Save the correlation coefficients for each fold
rs = np.zeros((nfolds,spectrogram.shape[1]))
for k,(train, test) in enumerate(kf.split(data)):
          
    #Train, validation and test data
    X_train_temp = data[train,:]
    y_train_temp = spectrogram[train,:]
    X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=val_split, random_state=0)
    X_test = data[test,:]
    y_test = spectrogram[test,:] # this one might not be needed
    
    #Normalization
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train[:] = scaler.transform(X_train)
    X_val[:] = scaler.transform(X_val)
    X_test[:] = scaler.transform(X_test)

    #Fit PCA to training data
    pca.fit(X_train)
    #Get percentage of explained variance by selected components
    explainedVariance[k] = np.sum(pca.explained_variance_ratio_[:numComps])
    #Tranform data 
    X_train = np.dot(X_train, pca.components_[:numComps,:].T)
    X_val = np.dot(X_val, pca.components_[:numComps,:].T)
    X_test = np.dot(X_test, pca.components_[:numComps,:].T)

    # Bottleneck model
    early_stopping=EarlyStopping(patience=25, verbose=1, min_delta=1e-5)
    checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=1)

    model = create_ae_model(numComps, spectrogram.shape[1])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.fit(X_train, y_train, batch_size=64, 
              epochs=500, verbose=0, validation_data=(X_val, y_val), shuffle=True,
              callbacks=[checkpointer, early_stopping])
            
    #predict with the Autoencoder
    model.load_weights('weights1.hdf5')
    rec_spec[test, :] = model.predict(X_test, verbose=0)

    #Evaluate reconstruction of this fold
    for specBin in range(spectrogram.shape[1]):
         r, p = pearsonr(spectrogram[test, specBin], rec_spec[test, specBin])
         rs[k,specBin] = r


#Show evaluation result
print('mean correlation', np.mean(rs))


Epoch 1: val_loss improved from inf to 5.08743, saving model to weights1.hdf5

Epoch 2: val_loss improved from 5.08743 to 3.29884, saving model to weights1.hdf5

Epoch 3: val_loss improved from 3.29884 to 2.55712, saving model to weights1.hdf5

Epoch 4: val_loss improved from 2.55712 to 2.20429, saving model to weights1.hdf5

Epoch 5: val_loss improved from 2.20429 to 2.04845, saving model to weights1.hdf5

Epoch 6: val_loss improved from 2.04845 to 1.91544, saving model to weights1.hdf5

Epoch 7: val_loss improved from 1.91544 to 1.71705, saving model to weights1.hdf5

Epoch 8: val_loss improved from 1.71705 to 1.50765, saving model to weights1.hdf5

Epoch 9: val_loss improved from 1.50765 to 1.50368, saving model to weights1.hdf5

Epoch 10: val_loss did not improve from 1.50368

Epoch 11: val_loss did not improve from 1.50368

Epoch 12: val_loss improved from 1.50368 to 1.47402, saving model to weights1.hdf5

Epoch 13: val_loss did not improve from 1.47402

Epoch 14: val_loss improv

KeyboardInterrupt: ignored

### Optimizing the LSTM structure for one person

In [4]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



data = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_feat.npy')
spectrogram = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_spec.npy')

#Inital parameters
nfolds = 10
kf = KFold(nfolds,shuffle=False)
pca = PCA()
numComps = 300
explainedVariance = np.zeros(nfolds)
val_split = 0.2


#Initialize an empty spectrogram to save the reconstruction to
rec_spec = np.zeros(spectrogram.shape)
#Save the correlation coefficients for each fold
rs = np.zeros((nfolds,spectrogram.shape[1]))
for k,(train, test) in enumerate(kf.split(data)):         
    #Train, validation and test data
    X_train_temp = data[train,:]
    y_train_temp = spectrogram[train,:]
    X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=val_split, random_state=0, shuffle=False)
    X_test = data[test,:]
    y_test = spectrogram[test,:] # this one might not be needed
    
    #Normalization
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train[:] = scaler.transform(X_train)
    X_val[:] = scaler.transform(X_val)
    X_test[:] = scaler.transform(X_test)

    #Fit PCA to training data
    pca.fit(X_train)
    #Get percentage of explained variance by selected components
    explainedVariance[k] = np.sum(pca.explained_variance_ratio_[:numComps])
    #Tranform data 
    X_train = np.dot(X_train, pca.components_[:numComps,:].T)
    X_val = np.dot(X_val, pca.components_[:numComps,:].T)
    X_test = np.dot(X_test, pca.components_[:numComps,:].T)

    # LSTM model
    early_stopping=EarlyStopping(patience=25, verbose=1, min_delta=1e-5)
    checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=1)

    model = create_LSTM_model(numComps, spectrogram.shape[1])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.fit(X_train, y_train, batch_size=64, 
              epochs=150, verbose=0, validation_data=(X_val, y_val), shuffle=True,
              callbacks=[checkpointer, early_stopping])
            
    #predict with the Autoencoder
    model.load_weights('weights1.hdf5')
    rec_spec[test, :] = model.predict(X_test, verbose=0)

    #Evaluate reconstruction of this fold
    for specBin in range(spectrogram.shape[1]):
         r, p = pearsonr(spectrogram[test, specBin], rec_spec[test, specBin])
         rs[k,specBin] = r


#Show evaluation result
print('mean correlation', np.mean(rs))


Epoch 1: val_loss improved from inf to 2.36893, saving model to weights1.hdf5

Epoch 2: val_loss did not improve from 2.36893

Epoch 3: val_loss did not improve from 2.36893

Epoch 4: val_loss did not improve from 2.36893

Epoch 5: val_loss did not improve from 2.36893

Epoch 6: val_loss did not improve from 2.36893

Epoch 7: val_loss did not improve from 2.36893

Epoch 8: val_loss did not improve from 2.36893

Epoch 9: val_loss did not improve from 2.36893

Epoch 10: val_loss did not improve from 2.36893

Epoch 11: val_loss did not improve from 2.36893

Epoch 12: val_loss did not improve from 2.36893

Epoch 13: val_loss did not improve from 2.36893

Epoch 14: val_loss did not improve from 2.36893

Epoch 15: val_loss did not improve from 2.36893

Epoch 16: val_loss did not improve from 2.36893

Epoch 17: val_loss did not improve from 2.36893

Epoch 18: val_loss did not improve from 2.36893

Epoch 19: val_loss did not improve from 2.36893

Epoch 20: val_loss did not improve from 2.3689

### Optimizing the CNN structure for one person

In [113]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



data = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_feat.npy')
spectrogram = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_spec.npy')

#Inital parameters
nfolds = 10
kf = KFold(nfolds,shuffle=False)
pca = PCA()
numComps = 200
explainedVariance = np.zeros(nfolds)
val_split = 0.2
window = 20


#Initialize an empty spectrogram to save the reconstruction to
rec_spec = np.zeros(spectrogram.shape)
#Save the correlation coefficients for each fold
rs = np.zeros((nfolds,spectrogram.shape[1]))
for k,(train, test) in enumerate(kf.split(data)):         
    #Train, validation and test data
    X_train_temp = data[train,:]
    y_train_temp = spectrogram[train,:]
    X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=val_split, random_state=0, shuffle=False)
    X_test = data[test,:]
    y_test = spectrogram[test,:] # this one might not be needed
    
    #Normalization
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train[:] = scaler.transform(X_train)
    X_val[:] = scaler.transform(X_val)
    X_test[:] = scaler.transform(X_test)

    #Fit PCA to training data
    pca.fit(X_train)
    #Get percentage of explained variance by selected components
    explainedVariance[k] = np.sum(pca.explained_variance_ratio_[:numComps])
    #Tranform data 
    X_train = np.dot(X_train, pca.components_[:numComps,:].T)
    X_val = np.dot(X_val, pca.components_[:numComps,:].T)
    X_test = np.dot(X_test, pca.components_[:numComps,:].T)

    #reshaping the data
    X_train = cnnize(X_train, window)
    X_val = cnnize(X_val, window)
    X_test = cnnize(X_test, window)

    # CNN model
    early_stopping=EarlyStopping(patience=25, verbose=1, min_delta=1e-5)
    checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=1)

    model = create_cnn_model(int((numComps-window)/2)+1, window, spectrogram.shape[1])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.fit(X_train, y_train, batch_size=64, 
              epochs=150, verbose=0, validation_data=(X_val, y_val), shuffle=True,
              callbacks=[checkpointer, early_stopping])
            
    #predict with the Autoencoder
    model.load_weights('weights1.hdf5')
    rec_spec[test, :] = model.predict(X_test, verbose=0)

    #Evaluate reconstruction of this fold
    for specBin in range(spectrogram.shape[1]):
         r, p = pearsonr(spectrogram[test, specBin], rec_spec[test, specBin])
         rs[k,specBin] = r


#Show evaluation result
print('mean correlation', np.mean(rs))


Epoch 1: val_loss improved from inf to 3.65074, saving model to weights1.hdf5

Epoch 2: val_loss improved from 3.65074 to 3.44781, saving model to weights1.hdf5

Epoch 3: val_loss improved from 3.44781 to 3.12265, saving model to weights1.hdf5

Epoch 4: val_loss did not improve from 3.12265

Epoch 5: val_loss improved from 3.12265 to 3.09337, saving model to weights1.hdf5

Epoch 6: val_loss improved from 3.09337 to 2.97692, saving model to weights1.hdf5

Epoch 7: val_loss improved from 2.97692 to 2.92251, saving model to weights1.hdf5

Epoch 8: val_loss did not improve from 2.92251

Epoch 9: val_loss did not improve from 2.92251

Epoch 10: val_loss did not improve from 2.92251

Epoch 11: val_loss improved from 2.92251 to 2.87664, saving model to weights1.hdf5

Epoch 12: val_loss did not improve from 2.87664

Epoch 13: val_loss did not improve from 2.87664

Epoch 14: val_loss did not improve from 2.87664

Epoch 15: val_loss did not improve from 2.87664

Epoch 16: val_loss did not impro

KeyboardInterrupt: ignored

# 2.) Trying it out for every subject

In [ ]:
import os

import numpy as np
import scipy.io.wavfile as wavfile
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

#import reconstructWave as rW
#import MelFilterBank as mel

# #Optional function
# def createAudio(spectrogram, audiosr=16000, winLength=0.05, frameshift=0.01):
#     mfb = mel.MelFilterBank(int((audiosr*winLength)/2+1), spectrogram.shape[1], audiosr)
#     nfolds = 10
#     hop = int(spectrogram.shape[0]/nfolds)
#     rec_audio = np.array([])
#     for_reconstruction = mfb.fromLogMels(spectrogram)
#     for w in range(0,spectrogram.shape[0],hop):
#         spec = for_reconstruction[w:min(w+hop,for_reconstruction.shape[0]),:]
#         rec = rW.reconstructWavFromSpectrogram(spec,spec.shape[0]*spec.shape[1],fftsize=int(audiosr*winLength),overlap=int(winLength/frameshift))
#         rec_audio = np.append(rec_audio,rec)
#     scaled = np.int16(rec_audio/np.max(np.abs(rec_audio)) * 32767)
#     return scaled


##### MODELING ########

model = True

scores= []

if model == True:
    feat_path = r'/content/drive/MyDrive/DeepLearning/features'
    result_path = r'/content/drive/MyDrive/DeepLearning/features'
    pts = ['sub-%02d'%i for i in range(1,11)]

    winLength = 0.05
    frameshift = 0.01
    audiosr = 16000

    nfolds = 15
    kf = KFold(nfolds,shuffle=False)
    pca = PCA()
    numComps = 400
    
    #Initialize empty matrices for correlation results, randomized controls and amount of explained variance
    allRes = np.zeros((len(pts),nfolds,23))
    explainedVariance = np.zeros((len(pts),nfolds))
    numRands = 1000
    randomControl = np.zeros((len(pts),numRands, 23))

    for pNr, pt in enumerate(pts):
        
        
        #Load the data
        #Dimensions of these data vary depending on the subject
        spectrogram = np.load(os.path.join(feat_path,f'{pt}_spec.npy'))  
        data = np.load(os.path.join(feat_path,f'{pt}_feat.npy'))
        labels = np.load(os.path.join(feat_path,f'{pt}_procWords.npy'))
        featName = np.load(os.path.join(feat_path,f'{pt}_feat_names.npy'))

        

        
        #Initialize an empty spectrogram to save the reconstruction to
        rec_spec = np.zeros(spectrogram.shape)
        #Save the correlation coefficients for each fold
        rs = np.zeros((nfolds,spectrogram.shape[1]))
        for k,(train, test) in enumerate(kf.split(data)):
          
            #Normalization
            mu=np.mean(data[train,:],axis=0)
            std=np.std(data[train,:],axis=0)
            trainData=(data[train,:]-mu)/std
            testData=(data[test,:]-mu)/std

            #Fit PCA to training data
            pca.fit(trainData)
            #Get percentage of explained variance by selected components
            explainedVariance[pNr,k] = np.sum(pca.explained_variance_ratio_[:numComps])
            #Tranform data into component space
            trainData = np.dot(trainData, pca.components_[:numComps,:].T)
            testData = np.dot(testData, pca.components_[:numComps,:].T)

            early_stopping=EarlyStopping(patience=25, verbose=0, min_delta=1e-5)
            checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=0)

            #Autoencoder
            model = create_ae_model(trainData.shape[1], spectrogram.shape[1])
            model.compile(loss='mse', optimizer='adam', metrics=['mse'])
            model.fit(trainData, spectrogram[train, :], batch_size=64, 
                      epochs=500, verbose=0, validation_split=1/9, shuffle=True,
                      callbacks=[checkpointer, early_stopping])
            
            #predict with the Autoencoder
            rec_spec[test, :] = model.predict(testData, verbose=0)

            #Evaluate reconstruction of this fold
            for specBin in range(spectrogram.shape[1]):
                if np.any(np.isnan(rec_spec)):
                    print('%s has %d broken samples in reconstruction' % (pt, np.sum(np.isnan(rec_spec))))
                r, p = pearsonr(spectrogram[test, specBin], rec_spec[test, specBin])
                rs[k,specBin] = r

        #Show evaluation result
        print('%s has mean correlation of %f' % (pt, np.mean(rs)))
        allRes[pNr,:,:]=rs
        scores.append(np.mean(rs))

#         #Estimate random baseline
#         for randRound in range(numRands):
#             #Choose a random splitting point at least 10% of the dataset size away
#             splitPoint = np.random.choice(np.arange(int(spectrogram.shape[0]*0.1),int(spectrogram.shape[0]*0.9)))
#             #Swap the dataset on the splitting point 
#             shuffled = np.concatenate((spectrogram[splitPoint:,:],spectrogram[:splitPoint,:]))
#             #Calculate the correlations
#             for specBin in range(spectrogram.shape[1]):
#                 if np.any(np.isnan(rec_spec)):
#                     print('%s has %d broken samples in reconstruction' % (pt, np.sum(np.isnan(rec_spec))))
#                 r, p = pearsonr(spectrogram[:,specBin], shuffled[:,specBin])
#                 randomControl[pNr, randRound,specBin]=r


#         #Save reconstructed spectrogram
#         os.makedirs(os.path.join(result_path), exist_ok=True)
#         np.save(os.path.join(result_path,f'{pt}_predicted_spec.npy'), rec_spec)
        
#         #Synthesize waveform from spectrogram using Griffin-Lim
#         reconstructedWav = createAudio(rec_spec,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
#         wavfile.write(os.path.join(result_path,f'{pt}_predicted.wav'),int(audiosr),reconstructedWav)

#         #For comparison synthesize the original spectrogram with Griffin-Lim
#         origWav = createAudio(spectrogram,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
#         wavfile.write(os.path.join(result_path,f'{pt}_orig_synthesized.wav'),int(audiosr),origWav)

#     #Save results in numpy arrays          
#     np.save(os.path.join(result_path,'linearResults.npy'),allRes)
#     np.save(os.path.join(result_path,'randomResults.npy'),randomControl)
#     np.save(os.path.join(result_path,'explainedVariance.npy'),explainedVariance)

KeyboardInterrupt: ignored

In [ ]:
scores

[0.5629875535958715,
 0.615317710223286,
 0.8574981748428402,
 0.816490556136459,
 0.5712122543230624,
 0.8819938942481294,
 0.7771201475020989,
 0.6878816278290169,
 0.7386449014724298,
 0.7146618900721041]